In [0]:
%sh
cd /Volumes/workspace/aerospace/flights/
unzip flights.zip
ls -al

In [0]:
# File location and type
file_location = "/Volumes/workspace/aerospace/flights/flights.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

In [0]:
df.write.saveAsTable('flights')


In [0]:
%sql
select * from `flights`

In [0]:
%sql
select * from aerospace.airlines